In [1]:
from urllib import parse, request
import hashlib
from datetime import date, timedelta
from time import strftime, gmtime
import pandas as pd
from pandas import DataFrame
from io import StringIO

In [33]:
my_merchant_id = 'xxx'
api_token = 'xxx'
api_secret_key = 'xxx'
my_timestamp = strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime())
api_version = 3.0
action_verb = 'transactiondetail'

date_start = date.today() - timedelta(days = 20)
date_end = date.today() - timedelta(days = 1)
export_type = 'pipe'
sortcol = 'transid'
sortdir = 'desc'

In [34]:
# remove dateStart and add sortCol and sortDir to get lifetime. # 'dateStart' : date_start,
data = parse.urlencode({'merchantId': my_merchant_id, 'token': api_token,
						'version': api_version, 'action': action_verb,
                    	'sortCol' : sortcol, 'sortDir' : sortdir, 
					   	'dateStart' : date_start, 'dateEnd' : date_end, 'format': export_type})

# authentication
sig = api_token + ':' + my_timestamp + ':' + action_verb + ':' + api_secret_key
sig_hash = hashlib.sha256(sig.encode('utf-8')).hexdigest()
my_headers = {'x-ShareASale-Date': my_timestamp,
				'x-ShareASale-Authentication': sig_hash}

In [35]:
call = request.Request('https://shareasale.com/w.cfm?%s' %
						data, headers=my_headers)
try:
	response = request.urlopen(call).read()
except Exception as inst:
	print(inst)

In [36]:
# decode output for df
output = response.decode("utf-8")

# push output into df
df = pd.read_csv(StringIO(output), sep='|', engine= 'python', 
usecols = ['transID','userID','transdate','transamount','commission','ssamount','comment','voided','locked',
        'pending','lastip','lastreferer','bannernumber','bannerpage','dateoftrans','dateofclick','timeofclick',
        'dateofreversal','returndays','toolID','storeID','lockDate','transactionType','CommissionType','skulist',
        'priceList','quantityList','orderNumber','parentTrans','bannerName','bannerType','couponCode','referenceTrans',
        'newCustomerFlag','userAgent','originalCurrency','originalCurrencyAmount','isMobile','usedACoupon','merchantDefinedType'])

# check counts/date range.         
df.shape
# min(df['transdate'])

(869, 40)

In [37]:
from pandasql import sqldf
import pyodbc

# For PandaSQL queries. Easier to share/explain steps.
pysqldf = lambda q: sqldf(q, globals())

# For ODBC driver.
drvr = '{ODBC Driver 17 for SQL Server}'
srvr = 'PNASQL01'
db = 'KP'
usr ='XXXXXXXX'
pw = 'XXXXXXXX'

In [38]:
# Orders With A Falsely Assigned New Customer Commission 

# df of "New Customers" according to Share-A-Sale.
newCust = pysqldf(
"SELECT ORDERNUMBER "
"FROM df "
"WHERE NEWCUSTOMERFLAG = 1 "    # Custom commission rate flag.
"AND VOIDED IS NULL "           # Not yet voided.
"AND LOCKED IS NULL "           # Safety parameter if this was/is run around the 20th.
"AND STOREID IN ('1','2')"      # Keeps stores to WC/KP. CT has it's own ipynb.
)

# handle float64 issue
newCust['orderNumber']  = newCust['orderNumber'].astype(int)

# Init ODBC connection.
connection = pyodbc.connect(DRIVER=drvr, SERVER=srvr, DATABASE=db, UID=usr, PWD=pw, Trusted_Connection='YES')

# Validate new customers against response server.
sql = (
    "SELECT A.ORDER_NO, A.WEB_ORDER_ID, A.ORDER_DATE, B.FIRST_PURCH "               # Extraneous columns to manually verify.
    "FROM KP.RESPONSE.SYSOENT A "                                                   # Order table.
    "INNER JOIN KP.RESPONSE.SYSCUST B "                                             # Customer table.
	"ON A.CUSTOMER_ID = B.CUSTOMER_ID "
    "WHERE CAST(A.ORDER_DATE AS DATE) > CAST(B.FIRST_PURCH AS DATE) "               # Compare order date to accounts first purchase.
    "AND WEB_ORDER_ID IN {}".format(tuple(str(x) for x in newCust['orderNumber']))  # Needed to dynamically pass df column as a string. 
)

# ODBC call.
cursor = connection.cursor()
response_newCust = pd.read_sql(sql, connection)

# List of order numbers to change commission rate.
commissionList = pysqldf(
"SELECT A.WEB_ORDER_ID, B.transdate "
"FROM response_newCust A "
"INNER JOIN df B ON A.WEB_ORDER_ID = B.orderNumber"
)

# total #
len(commissionList)

62

In [39]:
# Orders Containing Inelligible Items To Be Voided

# make new df for all order numbers, not just new cust
orderNos = pysqldf(
"SELECT ORDERNUMBER "
"FROM df "
"WHERE VOIDED IS NULL "             # Where not already voided.
"AND LOCKED IS NULL "               # Safety parameter if this was/is run around the 20th.
"AND STOREID IN ('1','2')"          # Keeps stores to WC/KP. CT has it's own ipynb.
)

# handle float64 issue
orderNos['orderNumber']  = orderNos['orderNumber'].astype(int)

# connection already set up, fire away
sql2 = (
    "SELECT A.ORDER_NO, A.WEB_ORDER_ID, A.ORDER_DATE "
    "FROM KP.RESPONSE.SYSOENT A "
    "WHERE ORDER_NO IN ( SELECT DISTINCT ORDER_NO "                                             # Easier to subquery, just pull distinct order numbers from itemized level.
    "FROM KP.RESPONSE.LINEITEM A INNER JOIN KP.RESPONSE.INVENTOR B ON A.ITEM_ID = B.ITEM_ID "
    "WHERE (B.PROD_CLASS_ID = 'PATTM' OR A.ITEM_ID = 'GIFT') "                                  # Check order contents for IDP designs and gift cards.
    "AND A.ORDER_DATE > GETDATE()-30 )"                                                         # Max window. Remove later.
    "AND A.WEB_ORDER_ID IN {}".format(tuple(str(x) for x in orderNos['orderNumber']))           # Dynamically pass order numbers as string from api call.
)


cursor = connection.cursor()
response_itemID = pd.read_sql(sql2, connection)

# df of commissions to be canceled later.
itemIDList = pysqldf(
"SELECT WEB_ORDER_ID, ORDER_DATE "
"FROM response_itemID "
)

# total #
len(itemIDList)

20

In [40]:
# Canceled Orders To Be Voided

# OrderNo df already exists.

# ODBC Connection already set up.

# Pull canceled orders.
sql3 = (
    "SELECT A.ORDER_NO, A.WEB_ORDER_ID, A.ORDER_DATE, A.ORDER_STATUS "                      # Extraneous columns to manually verify.
    "FROM KP.RESPONSE.SYSOENT A "                                                           # Order table.
    "WHERE A.ORDER_STATUS = 'X' "                                                           # Canceled orders status.
    "AND A.WEB_ORDER_ID IN {}".format(tuple(str(x) for x in orderNos['orderNumber']))       # Dynamically pass order numbers as string from api call.
)

# ODBC call.
cursor = connection.cursor()
response_canceled = pd.read_sql(sql3, connection)

# df of commissions to be canceled later.
canceledList = pysqldf(
"SELECT WEB_ORDER_ID, ORDER_DATE "
"FROM response_canceled"
)

# total #
len(canceledList)

0

In [27]:
# Definitions 
import time

def voidAPI(ordernumber, date):
    data = parse.urlencode({'merchantId': my_merchant_id, 'token': api_token, 'version': api_version, 'action': 'void', 'ordernumber': ordernumber, 'date': date, 'reason': 'Non-Fulfilled Requirements'})
    sig = api_token + ':' + my_timestamp + ':' + 'void' + ':' + api_secret_key
    sig_hash = hashlib.sha256(sig.encode('utf-8')).hexdigest()
    my_headers = {'x-ShareASale-Date': my_timestamp, 'x-ShareASale-Authentication': sig_hash}
    call = request.Request('https://shareasale.com/w.cfm?%s' % data, headers=my_headers)
    response = request.urlopen(call)
    return response


def editAPI(ordernumber, date):
    data = parse.urlencode({'merchantId': my_merchant_id, 'token': api_token, 'version': api_version, 'action': 'edit', 'ordernumber': ordernumber, 'date': date, 'newpercentage': '10.00', 'newcomment': 'Edit: Non-new customer. Commission rate reduced.'})
    sig = api_token + ':' + my_timestamp + ':' + 'edit' + ':' + api_secret_key
    sig_hash = hashlib.sha256(sig.encode('utf-8')).hexdigest()
    my_headers = {'x-ShareASale-Date': my_timestamp, 'x-ShareASale-Authentication': sig_hash}
    call = request.Request('https://shareasale.com/w.cfm?%s' % data, headers=my_headers)
    response = request.urlopen(call)
    return response

In [21]:
# Void orders containing ineligible items. 
for index, row in itemIDList.iterrows():
    voidAPI(row['WEB_ORDER_ID'], row['ORDER_DATE'])
    time.sleep(5) # Throws errors if too many pass to quickly.

In [32]:
# Reduce commission rates from 30% to 10% with validated non-new customer.
for index, row in commissionList.iterrows():
    editAPI(row['WEB_ORDER_ID'], row['transdate'])
    time.sleep(5) # Throws errors if too many pass to quickly.

In [ ]:
# Void commissions for canceled orders.
for index, row in canceledList.iterrows():
    voidAPI(row['WEB_ORDER_ID'], row['ORDER_DATE'])
    time.sleep(5) # Throws errors if too many pass to quickly.